In [1]:
pip install pyserial

Note: you may need to restart the kernel to use updated packages.


In [5]:
# Import necessary libraries
import cv2
import numpy as np
from tensorflow.keras.preprocessing import image
import tensorflow as tf
import serial
import time

# Set up the serial communication (adjust the COM port as per your system)
# For Windows: COM3, COM4, etc.
# For Linux/macOS: /dev/ttyUSB0, /dev/ttyACM0, etc.
ser = serial.Serial('COM7', 9600, timeout=1)
# time.sleep(2)  # Allow the connection to establish

# Load the trained model for stop/shoot detection
cnn = tf.keras.models.load_model('video_based_shoot_stop_new.h5')

# Define a function to preprocess and make predictions on hand regions
def predict_hand(frame, model):
    # Resize the frame to match the model's input size (64x64)
    resized_frame = cv2.resize(frame, (64, 64))
    
    # Convert the frame to an array and expand dimensions for model input
    frame_array = image.img_to_array(resized_frame)
    frame_array = np.expand_dims(frame_array, axis=0)
    
    # Make a prediction using the trained model
    result = model.predict(frame_array)
    
    # Return the prediction (stop or shoot)
    if result[0][0] == 1:
        return 'stop'
    else:
        return 'shoot'

# Function to detect the hand based on skin color
def detect_hand(frame):
    # Convert the frame to HSV color space for better color segmentation
    hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    # Define lower and upper bounds for skin color in HSV (adjust for your lighting)
    lower_skin = np.array([0, 20, 70], dtype=np.uint8)
    upper_skin = np.array([20, 255, 255], dtype=np.uint8)
    
    # Create a mask for skin color
    mask = cv2.inRange(hsv_frame, lower_skin, upper_skin)
    
    # Apply some morphological operations to remove noise
    mask = cv2.erode(mask, np.ones((3, 3), np.uint8), iterations=2)
    mask = cv2.dilate(mask, np.ones((3, 3), np.uint8), iterations=2)
    
    # Blur the mask to smooth it
    mask = cv2.GaussianBlur(mask, (5, 5), 100)
    
    # Find contours in the mask
    contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    # If any contours are found, assume the largest one is the hand
    if contours and len(contours) > 0:
        largest_contour = max(contours, key=cv2.contourArea)
        
        # Get the bounding box of the largest contour
        x, y, w, h = cv2.boundingRect(largest_contour)
        
        # Return the bounding box coordinates
        return (x, y, w, h), mask
    
    return None, mask

# Initialize video capture from the webcam
cap = cv2.VideoCapture(0)

# Check if the webcam is opened correctly
if not cap.isOpened():
    print("Error: Could not open video stream.")
    exit()

# Start capturing video in a loop
while True:
    # Capture frame-by-frame from the webcam
    ret, frame = cap.read()
    
    # If frame is read correctly, continue
    if not ret:
        print("Failed to grab frame")
        break

    # Detect the hand and get the bounding box
    hand_box, mask = detect_hand(frame)
    
    # If a hand is detected, draw a bounding box around it and classify the hand gesture
    if hand_box is not None:
        x, y, w, h = hand_box
        hand_region = frame[y:y+h, x:x+w]
        
        # Draw the bounding box around the detected hand
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        
        # Predict stop/shoot based on the hand region
        prediction = predict_hand(hand_region, cnn)
        print(prediction)
        pred = prediction
        # Send the prediction result to the Arduino via serial
        ser.write(pred.encode())  # Send 'stop' or 'shoot'
        time.sleep(1)
        # Display the prediction on the frame
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(frame, f'Prediction: {prediction}', (x, y - 10), font, 0.9, (255, 0, 0), 2, cv2.LINE_AA)
    
    # Display the original frame and the mask for debugging
    cv2.imshow('Real-time Stop/Shoot Detection', frame)
    cv2.imshow('Hand Mask', mask)
    
    # Break the loop when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture object and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()
ser.close()  # Close the serial connection when done


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
shoot
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
stop
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
shoot
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
shoot
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
stop
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
stop
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
stop
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
stop
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
stop
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
stop
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
stop
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
stop
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
stop
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
stop
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
stop
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
stop
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
stop
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
stop
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
stop
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
stop
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
stop
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
stop
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
stop
1/1 ━━━━━

SerialException: WriteFile failed (PermissionError(13, 'The device does not recognize the command.', None, 22))